In [5]:
# this code implements a deep neural network for feature learning
# this code comes from Vishwa's class on constructing a data loader and creating a model

# Some common system imports
import os
import sys
import importlib
import time
import csv

# Numeric computing
import numpy as np

# Sklearn functions are useful for generating train/test splits, and metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

from scipy.io import wavfile

# pytorch
import torch
import torch.utils.data as tdata

# Plotting (if we want it)
import matplotlib.pyplot as plt

# importing our own modules
import audio_datasets as ads

In [6]:
# loading the raw data for feature learning
training_path = os.path.join(os.getcwd(), "..", "training_data")
files = os.listdir(training_path)
wav_files = []
file_type = "wav"
LABELS = {"neutral": 0, "calm": 1, "happy": 2, "sad": 3, "angry": 4, "fearful": 5, "disgust": 6, "surprised": 7}

for file in files:
    curr_path = os.path.join(training_path, file)
    if os.path.isfile(curr_path) and file_type in file:
        for label in LABELS.keys():
            if label in file:
                wav_files.append((file, LABELS[label]))

data_array = []
label_array = []
for data in wav_files: #in the my_datasets code
    data_array.append(wavfile.read(os.path.join(training_path, data[0]))[1])
    label_array.append(data[1])

max_len = max([len(data) for data in data_array])
data_array = [np.resize(data, 48000*5) for data in data_array]

/tmp/ipykernel_6330/647700761.py:18: WavFileWarning: Chunk (non-data) not understood, skipping it.
  data_array.append(wavfile.read(os.path.join(training_path, data[0]))[1])


In [7]:
# loading the raw data for feature learning
test_path = os.path.join(os.getcwd(), "..", "test_data")
test_files = os.listdir(test_path)
test_wav_files = []

for file in test_files:
    curr_path = os.path.join(test_path, file)
    if os.path.isfile(curr_path) and file_type in file:
        test_wav_files.append(file)

test_data_array = []
for data in test_wav_files: #in the my_datasets code
    test_data_array.append(wavfile.read(os.path.join(test_path, data))[1])

test_data_array = [np.resize(data, 48000*5) for data in test_data_array]

/tmp/ipykernel_6330/2716096835.py:13: WavFileWarning: Chunk (non-data) not understood, skipping it.
  test_data_array.append(wavfile.read(os.path.join(test_path, data))[1])


In [8]:
forest = RandomForestClassifier(1000) #gnb and y_pred lines directly from scikit-learn website documentation for Naive-Bayes
y_pred = forest.fit(data_array, label_array).predict(test_data_array)

LABEL_CONVERTER = dict([(value, key) for key, value in LABELS.items()])

#counting the mislabeled points
with open("random_trees_results.csv", "w+") as result_file:
   writer = csv.writer(result_file)
   writer.writerow(["filename", "label"])

   for idx in range(len(y_pred)):
       writer.writerow([os.path.splitext(test_wav_files[idx])[0], LABEL_CONVERTER[y_pred[idx]]])

#print(classification_report(test_y_ten, test_pred))
#print("Number of mislabeled points out of a total %d points : %d", (test_ten.shape[0], (test_y_ten != y_pred).sum()))